In [1]:
import numpy as np
from numba import njit

## nb_permutations

In [2]:
@njit(error_model="numpy")
def nb_permutations(pattern, length):
    # pattern :1d ndarray, likely np.arange(1,10,2)
    # length :int, length<=pattern.shape[0]
    loop = 1
    n_pattern = pattern.shape[0]
    for i in range(length):
        loop *= n_pattern - i
    return_arr = np.zeros((loop, length), dtype="int8")
    list_index_pattern = np.arange(n_pattern)
    for i in range(loop):
        mask = np.ones(n_pattern, dtype="bool")
        num = i
        for j in range(length):
            index_pattern = (list_index_pattern[mask])[num % (n_pattern - j)]
            return_arr[i, j] = pattern[index_pattern]
            num //= n_pattern - j
            mask[index_pattern] = False
    return return_arr

In [7]:
nb_permutations(np.arange(1,8,2), 2)

array([[1, 3],
       [3, 1],
       [5, 1],
       [7, 1],
       [1, 5],
       [3, 5],
       [5, 3],
       [7, 3],
       [1, 7],
       [3, 7],
       [5, 7],
       [7, 5]], dtype=int8)

## make_dict_change_x_pattern

In [3]:
def make_dict_change_x_pattern(max_ops):
    dict_change_x_pattern = dict()
    dict_change_x_pattern[0] = np.zeros((1, 1), dtype="int8")
    dict_max_loop = np.ones((max_ops + 1, max_ops + 1), dtype="int64")
    for i in range(1, max_ops + 1):
        change_x_pattern = nb_permutations(np.arange(1, max_ops + 2), i)  # not use 0,it is const(1)
        max_drop_num = np.zeros(change_x_pattern.shape[0], dtype="int8")
        for j in range(change_x_pattern.shape[0]):
            max_ = change_x_pattern[j, 0]
            save = max_ - 1
            for k in range(1, change_x_pattern.shape[1]):
                if max_ + 1 == change_x_pattern[j, k]:
                    max_ = change_x_pattern[j, k]
                elif max_ + 1 < change_x_pattern[j, k]:
                    max_ = change_x_pattern[j, k]
                    save = max_ - 1
            max_drop_num[j] = save
        dict_change_x_pattern[i] = change_x_pattern[np.argsort(max_drop_num)].astype("int8")
        dict_max_loop[i] = np.array([np.sum(max_drop_num <= j) for j in range(np.max(max_drop_num) + 1)])
    return dict_change_x_pattern, dict_max_loop

In [9]:
dict_change_x_pattern, dict_max_loop=make_dict_change_x_pattern(4)
#dict_change_x_pattern

In [14]:
def make_p(n,a,p=1/2):
    return_p = p**a*(1-p)**(n-a)
    for i in range(a):
        return_p*=((n-i)/(1+i))
    print(f"{int(return_p*10000)/100}%")

make_p(10,8)

4.39%


In [10]:
#dict_max_loop

In [11]:
n=3
print(f"0 : {dict_change_x_pattern[n][:dict_max_loop[n,0]]}")
for i in range(1,dict_max_loop.shape[0]):
    print(f"{i} : {dict_change_x_pattern[n][dict_max_loop[n,i-1]]}~{dict_change_x_pattern[n][dict_max_loop[n,i]-1]}")

0 : [[1 2 3]]
1 : [2 1 3]~[2 3 1]
2 : [1 3 4]~[3 1 2]
3 : [4 1 3]~[4 2 1]
4 : [1 3 5]~[5 4 3]


## make_change_x_id

In [5]:
def make_change_x_id(mask, x_max):
    # mask -> [3,4,2] みたいな
    # x_max -> 全体としての最大値
    if np.sum(mask) == 0:
        return 0
    TF = np.ones(x_max, dtype="bool")
    return_num = 0
    for i in range(mask.shape[0]):
        return_num *= x_max - i
        return_num += np.sum(TF[: mask[i] - 1])
        TF[mask[i] - 1] = False
    return return_num

## make_dict_mask_x

In [6]:
def make_dict_mask_x(eq_x_max):
    dict_mask_x = dict()
    for i in range(eq_x_max + 1):
        __mask_x = nb_permutations(np.arange(1, i + 1), i)  # 第一要素が0にする。
        mask_x = np.zeros((__mask_x.shape[0], __mask_x.shape[1] + 1), dtype="int8")
        mask_x[:, 1:] = __mask_x  # 第一要素が0にする。
        change_x_num = np.array([make_change_x_id(mask_x[k, 1:], i) for k in range(mask_x.shape[0])])
        index = np.argsort(change_x_num)
        dict_mask_x[i] = mask_x[index].copy()
    return dict_mask_x
make_dict_mask_x(3)

{0: array([[0]], dtype=int8),
 1: array([[0, 1]], dtype=int8),
 2: array([[0, 1, 2],
        [0, 2, 1]], dtype=int8),
 3: array([[0, 1, 2, 3],
        [0, 1, 3, 2],
        [0, 2, 1, 3],
        [0, 2, 3, 1],
        [0, 3, 1, 2],
        [0, 3, 2, 1]], dtype=int8)}

In [14]:
eq_x_max=3
dict_mask_x=make_dict_mask_x(eq_x_max)
for i in range(eq_x_max+1):
    for j in range(dict_mask_x[i].shape[0]):
        print(f"{i}: {make_change_x_id(dict_mask_x[i][j,1:],i)}:{dict_mask_x[i][j]}")

0: 0:[0]
1: 0:[0 1]
2: 0:[0 1 2]
2: 1:[0 2 1]
3: 0:[0 1 2 3]
3: 1:[0 1 3 2]
3: 2:[0 2 1 3]
3: 3:[0 2 3 1]
3: 4:[0 3 1 2]
3: 5:[0 3 2 1]


In [7]:
import numpy as np
from numba import njit
import timeit

@njit(error_model="numpy")
def count_True(arr, mode, border):
    count = 0
    match mode:
        case 0:  # lambda x: x
            for i in arr.flat:
                if i:
                    count += 1
        case 1:  # lambda x: x == border
            for i in arr.flat:
                if i == border:
                    count += 1
        case 2:  # lambda x: x != border
            for i in arr.flat:
                if i != border:
                    count += 1
        case 3:  # lambda x: x < border
            for i in arr.flat:
                if i < border:
                    count += 1
        case 4:  # lambda x: x <= border
            for i in arr.flat:
                if i <= border:
                    count += 1
        case 5:  # lambda x: x > border
            for i in arr.flat:
                if i > border:
                    count += 1
        case 6:  # lambda x: x >= border
            for i in arr.flat:
                if i >= border:
                    count += 1
    return count

@njit(error_model="numpy")
def nb_calc_RPN_1(x, equation):
    int_nan = -100
    n_sample = x.shape[1]
    stack = np.empty((count_True(equation, 6, 0), n_sample), dtype="float64")  # 6 -> lambda x: x >= border
    next_index = 0
    for i in range( equation.shape[0]):
        last_op = equation[i]
        if last_op >= 0:
            for j in range(n_sample):
                stack[next_index,j] = x[last_op,j]
            next_index += 1
        elif last_op == int_nan:
            break
        else:
            calc_arr_binary_op(last_op, stack[next_index - 2], stack[next_index - 1])
            next_index -= 1
    return_stack = np.empty((2, n_sample), dtype="float64")
    _mean = np.mean(stack[0])
    plus_minus_0 = np.sign(stack[0, 0] - _mean)
    plus_minus_1 = np.sign(stack[0, 0])
    sum = 0.
    for i in range(n_sample):
        sum += (stack[0,i] - _mean) ** 2
    std = np.sqrt(sum/n_sample)
    for i in range(n_sample):
        return_stack[0,i] = plus_minus_0 * (stack[0, i] - _mean) / std
        return_stack[1,i] = plus_minus_1 * stack[0, i]
    return return_stack

@njit(error_model="numpy")  # ,fastmath=True)
def set_x(arr_in, arr_out):
    for i in range(arr_in.shape[0]):
        arr_in[i] = arr_out[i]

@njit(error_model="numpy")  # ,fastmath=True)
def calc_arr_binary_op(op, arr1, arr2):
    match op:
        case -1:  # +
            for i in range(arr1.shape[0]):
                arr1[i] += arr2[i]
        case -2:  # -
            for i in range(arr1.shape[0]):
                arr1[i] -=  arr2[i]
        case -3:  # *
            for i in range(arr1.shape[0]):
                arr1[i] *=  arr2[i]
        case -4:  # a / b  (a > b)
            for i in range(arr1.shape[0]):
                arr1[i] /= arr2[i]
        case -5:  # b / a  (a > b)
            for i in range(arr1.shape[0]):
                arr1[i] = arr2[i] / arr1[i]


x = np.random.random((10,300))
#equation = np.array([2,1,5,6,3,-3,-2,-4,-5])
#equation = np.array([2,1,-3,5,-2,6,-4,3,-5])
equation = np.array([2,1,-3,5,-2,6,-4,3,-3])
print(nb_calc_RPN_1(x, equation)[1,0])
loop = 1000000
print(timeit.timeit(lambda: nb_calc_RPN_1(x, equation), number=loop))

0.17736322591645937
1.5943473340012133


In [8]:
import array
import numpy as np
from numba import njit
import timeit

@njit(error_model="numpy")
def isclose(a, b, rtol=1e-06, atol=0):
    return np.abs(a - b) <= atol + rtol * np.abs(b)

@njit(error_model="numpy")
def isclose_arr_1(arr1, arr2, rtol=1e-06, atol=0):
    for i in range(arr1.shape[0]):
        if not isclose(arr1[i], arr2[i], rtol=rtol, atol=atol):
            return False
    return True

@njit(error_model="numpy")
def isclose_arr_2(arr1, arr2, rtol=1e-06, atol=0):
    if np.all(isclose(arr1, arr2, rtol=rtol, atol=atol)):
        return True
    return False

arr1 = np.random.random((300))
arr2 = np.random.random((300))

print(isclose_arr_1(arr1, arr2),isclose_arr_2(arr1, arr2))
loop = 1000000
print(timeit.timeit(lambda: isclose_arr_1(arr1, arr2), number=loop))
print(timeit.timeit(lambda: isclose_arr_2(arr1, arr2), number=loop))

False False
0.2883642920060083
0.5032250420190394


[[ 1  2 -1  3 -3  1  4 -3 -1]
 [ 1  2 -1  3 -3  2  4 -3 -1]
 [ 1  2 -1  3 -3  4  2 -3 -1]
 [ 1  2 -1  3 -3  4  1 -3 -1]]
[[ True  True  True  True  True  True False False False False False False]
 [False False False False False  True  True  True False  True False False]
 [False False False  True False False False False  True False  True  True]
 [False False False False False False False False  True  True False False]]
 [[   1    3]
  [   1    2]
  [   1    4]]

 [[   1    4]
  [   1    2]
  [   2    4]]

 [[   1    4]
  [   1    2]
  [   4    2]]

In [17]:
max_ops = 4
dict_mask_x=make_dict_mask_x(max_ops)
dict_mask_x[4]

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 4, 3],
       [0, 1, 3, 2, 4],
       [0, 1, 3, 4, 2],
       [0, 1, 4, 2, 3],
       [0, 1, 4, 3, 2],
       [0, 2, 1, 3, 4],
       [0, 2, 1, 4, 3],
       [0, 2, 3, 1, 4],
       [0, 2, 3, 4, 1],
       [0, 2, 4, 1, 3],
       [0, 2, 4, 3, 1],
       [0, 3, 1, 2, 4],
       [0, 3, 1, 4, 2],
       [0, 3, 2, 1, 4],
       [0, 3, 2, 4, 1],
       [0, 3, 4, 1, 2],
       [0, 3, 4, 2, 1],
       [0, 4, 1, 2, 3],
       [0, 4, 1, 3, 2],
       [0, 4, 2, 1, 3],
       [0, 4, 2, 3, 1],
       [0, 4, 3, 1, 2],
       [0, 4, 3, 2, 1]], dtype=int8)

In [27]:
def nb_calc_RPN(x, equation):
    int_nan = -100
    n_sample = x.shape[1]
    stack = np.empty((count_True(equation, 6, 0), n_sample), dtype="float64")  # 6 -> lambda x: x >= border
    next_index = 0
    for i in range(equation.shape[0]):
        last_op = equation[i]
        if last_op >= 0:
            set_x(stack[next_index], x[last_op])
            next_index += 1
        elif last_op == int_nan:
            break
        else:
            calc_arr_binary_op(last_op, stack[next_index - 2], stack[next_index - 1])
            next_index -= 1
    return_stack = np.empty((2, n_sample), dtype="float64")
    if is_inf(stack[0]):
        return_stack[:] = int_nan
        return return_stack
    if is_all_zero(stack[0]):
        return_stack[0] = int_nan
        return_stack[1] = 0
        return return_stack
    if is_all_const(stack[0]):
        return_stack[0] = int_nan
        return_stack[1] = np.sign(stack[0, 0]) * stack[0]
        return return_stack
    _mean = np.mean(stack[0])
    plus_minus_0 = np.sign(stack[0, 0] - _mean)
    plus_minus_1 = np.sign(stack[0, 0])
    sum = 0.0
    for i in range(n_sample):
        sum += (stack[0, i] - _mean) ** 2
    std = np.sqrt(sum / n_sample)
    for i in range(n_sample):
        return_stack[0, i] = plus_minus_0 * (stack[0, i] - _mean) / std
        # return_stack[1, i] = stack[0, i]
        return_stack[1, i] = plus_minus_1 * stack[0, i]
    return return_stack


def set_x(arr_in, arr_out):
    for i in range(arr_in.shape[0]):
        arr_in[i] = arr_out[i]


def calc_arr_binary_op(op, arr1, arr2):
    match op:
        case -1:  # +
            for i in range(arr1.shape[0]):
                arr1[i] += arr2[i]
        case -2:  # -
            for i in range(arr1.shape[0]):
                arr1[i] -= arr2[i]
        case -3:  # *
            for i in range(arr1.shape[0]):
                arr1[i] *= arr2[i]
        case -4:  # a / b
            for i in range(arr1.shape[0]):
                arr1[i] /= arr2[i]


def isclose(a, b, rtol=1e-06, atol=0):
    return np.abs(a - b) <= atol + rtol * np.abs(b)


def is_inf(arr):
    for i in range(arr.shape[0]):
        if np.isinf(arr[i]):
            return True
    return False

def is_all_zero(arr, atol=1e-06):
    for i in range(arr.shape[0]):
        if not isclose(0, arr[i], rtol=0, atol=atol):
            return False
    return True


def is_all_const(arr, rtol=1e-06, atol=0):
    min, max, sum = arr[0], arr[0], arr[0]
    for i in range(1, arr.shape[0]):
        if min > arr[i]:
            min = arr[i]
        if max < arr[i]:
            max = arr[i]
        sum += arr[i]
    return (max - min) <= atol + rtol * np.abs(sum / arr.shape[0])


def isclose_arr(arr1, arr2, rtol=1e-06, atol=0):
    # if np.all(isclose(arr1, arr2, rtol=rtol, atol=atol)):
    #    return True
    for i in range(arr1.shape[0]):
        if not isclose(arr1[i], arr2[i], rtol=rtol, atol=atol):
            return False
    return True

def make_check_change_x(eq, mask, same_arr):
    int_nan = -100
    if np.max(same_arr) == same_arr.shape[0] - 1:
        return np.empty((0, 2), dtype="int8")
    max_same_arr = np.max(same_arr)
    len_arr = mask.shape[1]
    len_return_arr = ((len_arr - 2) * (len_arr - 1)) // 2
    TF = np.empty((len_return_arr, mask.shape[0]), dtype="bool")
    return_arr = np.empty((len_return_arr, 2), dtype="int8")
    n = 0
    for i in range(1, len_arr - 1):
        for j in range(i + 1, len_arr):
            return_arr[n, 0] = i
            return_arr[n, 1] = j
            n += 1
    for i in range(len_return_arr):
        for j in range(mask.shape[0]):
            TF[i,j] = mask[j, return_arr[i,0]] < mask[j, return_arr[i,1]]

    use = np.zeros((len_return_arr), dtype="bool")
    Reduction_ratio = mask.shape[0] // (max_same_arr+1)
    tot_TF = np.empty((mask.shape[0]), dtype="bool")
    saved_num = np.empty((max_same_arr+1), dtype="bool")
    print(Reduction_ratio,mask.shape[0],max_same_arr+1)
    p = 1
    min_use = int_nan
    for i in range(2,len_return_arr + 2):
        p *= i
        if Reduction_ratio <= p:
            min_use = i - 1
            break
    max_use = int(np.log2(Reduction_ratio))+int(np.sign(np.log2(Reduction_ratio)%1))
    for i in range(1, max_use + 1):
        divide_num = len_return_arr-(i-1)
        for j in range(divide_num**i):
            num = j
            max_setted_num = -1
            use[:] = False
            check = True
            for k in range(i):
                set_num = num%divide_num + k
                num //= divide_num
                if max_setted_num < set_num:
                    max_setted_num = set_num
                    use[max_setted_num] = True
                else:
                    check = False
                    break
            if check:
                can_use = True
                tot_TF[:] = True
                saved_num[:] = False
                for k in range(len_return_arr):
                    if use[k]:
                        for l in range(mask.shape[0]):
                            if not TF[k,l]:
                                tot_TF[l] = False
                for k in range(mask.shape[0]):
                    if tot_TF[k]:
                        if saved_num[same_arr[k]]:
                            can_use =False
                            break
                        else:
                            saved_num[same_arr[k]] = True
                print(return_arr[use])
                print(can_use,saved_num)
                for t in range(mask.shape[0]):
                    if tot_TF[t]:
                        print(mask[t],same_arr[t])
                if can_use:
                    if np.all(saved_num):
                        return return_arr[use]
    print("ERROR : not seleced")


def check(eq,len_random_x=20):
    max_op = eq.shape[0]//2
    random_x = np.random.random((max_op+2, len_random_x))
    random_x[0] = 1
    tot_mask_x = nb_permutations(np.arange(max_op + 1), max_op + 1)
    eq_x_max = np.max(eq)
    mask = make_dict_mask_x(max_op + 1)[eq_x_max]
    same_for_mask_x = np.zeros((tot_mask_x.shape[0]), dtype="int64")
    cache_for_mask_x = np.zeros((tot_mask_x.shape[0], len_random_x), dtype="float64")
    cache_for_mask_x[0] = nb_calc_RPN(random_x, eq)[1]
    same_for_mask_x[0] = 0
    c = 1
    for k in range(1, mask.shape[0]):
        similar_num = nb_calc_RPN(random_x[mask[k]], eq)
        same = False
        for l in range(c):
            if isclose(cache_for_mask_x[l, 0], similar_num[1, 0]):
                if isclose_arr(cache_for_mask_x[l], similar_num[1]):
                    same_for_mask_x[k] = l
                    same = True
        if not same:
            same_for_mask_x[k] = c
            cache_for_mask_x[c] = similar_num[1]
            c += 1
    same_arr = same_for_mask_x[: mask.shape[0]]
    return make_check_change_x(eq, mask, same_arr)
eq = np.array([1, 2, -2, 3, 4, -2, -3])
check(eq)

8 24 3
[[1 2]]
False [ True False False]
[0 1 2 3 4] 0
[0 1 2 4 3] 0
[0 1 3 2 4] 1
[0 1 3 4 2] 1
[0 1 4 2 3] 2
[0 1 4 3 2] 2
[0 2 3 1 4] 2
[0 2 3 4 1] 2
[0 2 4 1 3] 1
[0 2 4 3 1] 1
[0 3 4 1 2] 0
[0 3 4 2 1] 0
[[1 3]]
False [ True False False]
[0 1 2 3 4] 0
[0 1 2 4 3] 0
[0 1 3 2 4] 1
[0 1 3 4 2] 1
[0 1 4 2 3] 2
[0 1 4 3 2] 2
[0 2 1 3 4] 0
[0 2 1 4 3] 0
[0 2 3 4 1] 2
[0 2 4 3 1] 1
[0 3 1 4 2] 1
[0 3 2 4 1] 2
[[1 4]]
False [ True False False]
[0 1 2 3 4] 0
[0 1 2 4 3] 0
[0 1 3 2 4] 1
[0 1 3 4 2] 1
[0 1 4 2 3] 2
[0 1 4 3 2] 2
[0 2 1 3 4] 0
[0 2 1 4 3] 0
[0 2 3 1 4] 2
[0 2 4 1 3] 1
[0 3 1 2 4] 1
[0 3 2 1 4] 2
[[2 3]]
False [ True False False]
[0 1 2 3 4] 0
[0 1 2 4 3] 0
[0 1 3 4 2] 1
[0 2 1 3 4] 0
[0 2 1 4 3] 0
[0 2 3 4 1] 2
[0 3 1 2 4] 1
[0 3 1 4 2] 1
[0 3 2 4 1] 2
[0 4 1 2 3] 2
[0 4 1 3 2] 2
[0 4 2 3 1] 1
[[2 4]]
False [ True False False]
[0 1 2 3 4] 0
[0 1 2 4 3] 0
[0 1 3 2 4] 1
[0 2 1 3 4] 0
[0 2 1 4 3] 0
[0 2 3 1 4] 2
[0 3 1 2 4] 1
[0 3 1 4 2] 1
[0 3 2 1 4] 2
[0 4 1 2 3] 2
[0 4 1 3 2]

array([[1, 2],
       [1, 3],
       [3, 4]], dtype=int8)

In [27]:
eq = np.array([1,2,3,-4,-4,4,-2,2,-3])
check(eq)

8 24 3


array([[1, 2],
       [1, 3],
       [2, 4]], dtype=int8)

In [29]:
eq = np.array([ 1,2,-2,3,4,-2,-3])
check(eq)

8 24 3


array([[1, 2],
       [1, 3],
       [3, 4]], dtype=int8)

In [28]:
eq = np.array([ 1,2,-2,3,4,-2,-3,5,-2])
check(eq)

4 120 30
[[1 2]]
False [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False False False False False False]
[0 1 2 3 4 5] 0
[0 1 2 3 5 4] 1
[0 1 2 4 3 5] 2
[0 1 2 4 5 3] 3
[0 1 2 5 3 4] 4
[0 1 2 5 4 3] 5
[0 1 3 2 4 5] 6
[0 1 3 2 5 4] 7
[0 1 3 4 2 5] 8
[0 1 3 4 5 2] 9
[0 1 3 5 2 4] 10
[0 1 3 5 4 2] 11
[0 1 4 2 3 5] 12
[0 1 4 2 5 3] 13
[0 1 4 3 2 5] 14
[0 1 4 3 5 2] 15
[0 1 4 5 2 3] 16
[0 1 4 5 3 2] 17
[0 1 5 2 3 4] 18
[0 1 5 2 4 3] 19
[0 1 5 3 2 4] 20
[0 1 5 3 4 2] 21
[0 1 5 4 2 3] 22
[0 1 5 4 3 2] 23
[0 2 3 1 4 5] 12
[0 2 3 1 5 4] 18
[0 2 3 4 1 5] 14
[0 2 3 4 5 1] 24
[0 2 3 5 1 4] 20
[0 2 3 5 4 1] 25
[0 2 4 1 3 5] 6
[0 2 4 1 5 3] 19
[0 2 4 3 1 5] 8
[0 2 4 3 5 1] 26
[0 2 4 5 1 3] 22
[0 2 4 5 3 1] 27
[0 2 5 1 3 4] 7
[0 2 5 1 4 3] 13
[0 2 5 3 1 4] 10
[0 2 5 3 4 1] 28
[0 2 5 4 1 3] 16
[0 2 5 4 3 1] 29
[0 3 4 1 2 5] 0
[0 3 4 1 5 2] 21
[0 3 4 2 1 5] 2
[0 3 4 2 5 1] 28
[0 3 4 5 1 2] 23
[0 3 4 5 

In [21]:
import numpy as np
Reduction_ratio = 1.5
max_use = int(np.log2(Reduction_ratio)) + int(np.sign(np.log2(Reduction_ratio) % 1))

In [13]:
len_return_arr = 10
i = 4
combination = 1.0
for j in range(i):
    combination *= ((len_return_arr - j)/(j+1))
combination = int(combination + 0.01)
combination

210

In [12]:
10*9*8*7/1/2/3/4

210.0

In [24]:
def func(len_return_arr):
    arange=np.arange(len_return_arr)
    use = np.zeros((len_return_arr), dtype="bool")
    for i in range(1, len_return_arr + 1):
        divide_num = len_return_arr-(i-1)
        for j in range(divide_num**i):
            num = j
            max_setted_num = -1
            use[:] = False
            check = True
            for k in range(i):
                set_num = num%divide_num + k
                num //= divide_num
                if max_setted_num < set_num:
                    max_setted_num = set_num
                    use[max_setted_num] = True
                else:
                    check = False
                    break
            if check:
                print(arange[use],j,num)
len_return_arr = 4
func(len_return_arr)


[0] 0 0
[1] 1 0
[2] 2 0
[3] 3 0
[0 1] 0 0
[0 2] 3 0
[1 2] 4 0
[0 3] 6 0
[1 3] 7 0
[2 3] 8 0
[0 1 2] 0 0
[0 1 3] 4 0
[0 2 3] 6 0
[1 2 3] 7 0
[0 1 2 3] 0 0


In [ ]:
def func(len_return_arr):
    arange=np.arange(len_return_arr)
    use = np.zeros((len_return_arr), dtype="bool")
    for i in range(1, len_return_arr + 1):
        combination = 1.0
        for j in range(i):
            combination *= ((len_return_arr - j)/(j + 1))
        combination = int(combination + 0.01)
        for j in range(combination):
            num = j
            max_setted_num = -1
            use[:] = False
            for k in range(i):
                divide_num = (len_return_arr-(i-1-k)-(max_setted_num+1))
                set_num = num%divide_num
                num //= divide_num
                max_setted_num = max_setted_num +1 + set_num
                use[max_setted_num] = True
            print(arange[use],j,num)
len_return_arr = 3
func(len_return_arr)


In [18]:
import numpy as np
np.expand_dims(np.arange(4).reshape(2,2),axis = 0).shape

(1, 2, 2)